In [ ]:
# Empirical Price Analysis
import csv
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

def load_eth_price_data():
    with open("../data/weth_usdc_005_tier_trades.csv","r") as infile:
        data = pd.read_csv(infile)

    with open("../data/weth_usdc_005_tier_trades2.csv","r") as infile:
        data2 = pd.read_csv(infile)

    data3 = pd.concat([data,data2]).drop_duplicates()
    data3["WETH"]=data3["amount1"].apply(lambda x: int(x) / 1e18)
    data3["USDC"]=data3["amount0"].apply(lambda x: int(x)/1e6)
    data3["price_implied"] = data3["price_implied"]/1e18
    data3["price_actual"] = data3["price_actual"]/1e18
    data3["price_impact"]=abs(data3["price_implied"]-data3["price_actual"])/data3["price_implied"]
    data3["evt_block_time"] = pd.to_datetime(data3['evt_block_time'], utc=True)
    data3["hourly_blocktime"] = data3["evt_block_time"].dt.floor("H")
    data3["daily_blocktime"] = data3["evt_block_time"].dt.floor("D")
    data3['unix_timestamp'] = data3['evt_block_time'].astype(int) // 10**9
    data3.sort_values(by="unix_timestamp", axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last')
    data3["hourly_timestamp"] = data3["hourly_blocktime"].astype(int) // 10**9
    data3["daily_timestamp"] = data3["daily_blocktime"].astype(int) // 10**9
    return(data3)

def load_stable_data():
    with open("../data/usdc_usdt_001_tier_trades.csv","r") as infile:
        data = pd.read_csv(infile)

    data["USDC"]=data["amoun0_adjusted"].apply(lambda x: int(x))
    data["USDT"]=data["amount1_adjusted"].apply(lambda x: int(x))

    data["price_implied"] = data["price_implied"]
    data["price_actual"] = data["price_actual"]
    data["price_impact"]=abs(data["price_implied"]-data["price_actual"])/data["price_implied"]
    data["evt_block_time"] = pd.to_datetime(data['evt_block_time'], utc=True)
    data["hourly_blocktime"] = data["evt_block_time"].dt.floor("H")
    data["daily_blocktime"] = data["evt_block_time"].dt.floor("D")
    data['unix_timestamp'] = data['evt_block_time'].astype(int) // 10**9
    data.sort_values(by="unix_timestamp", axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last')
    data["hourly_timestamp"] = data["hourly_blocktime"].astype(int) // 10**9
    data["daily_timestamp"] = data["daily_blocktime"].astype(int) // 10**9
    return(data)

FileNotFoundError: [Errno 2] No such file or directory: '../data/weth_usdc_005_tier_trades.csv'

In [ ]:
df = load_eth_price_data()

daily_price_data = df.groupby("daily_timestamp").agg({"price_actual": ["min","max","mean",
    ("first", lambda x: x.iloc[0]),("last", lambda x: x.iloc[-1])]})
daily_price_data["drops"]=(daily_price_data["price_actual"]["max"]-daily_price_data["price_actual"]["min"])/daily_price_data["price_actual"]["first"]
daily_price_data["drops_bin"] = (daily_price_data["drops"]*1000).astype(int)/1000

counts,bin_edges = np.histogram(daily_price_data["drops"],bins=100)
hist_daily_df = pd.DataFrame({"Bin_Left_Edge":bin_edges[:-1],"Bin_Reft_Edge":bin_edges[1:],"Count":counts})

hourly_price_data = df.groupby("hourly_timestamp").agg({"price_actual": ["min","max","mean",
    ("first", lambda x: x.iloc[0]),("last", lambda x: x.iloc[-1])]})
hourly_price_data["drops"]=(hourly_price_data["price_actual"]["max"]-hourly_price_data["price_actual"]["min"])/hourly_price_data["price_actual"]["first"]
hourly_price_data["drops_bin"] = (hourly_price_data["drops"]*1000).astype(int)/1000

counts,bin_edges = np.histogram(hourly_price_data["drops"],bins=100)
hist_hourly_df = pd.DataFrame({"Bin_Left_Edge":bin_edges[:-1],"Bin_Reft_Edge":bin_edges[1:],"Count":counts})

print(hist_daily_df,hist_daily_df["Count"].sum())
fig = px.scatter(x=daily_price_data.index, y=daily_price_data["drops_bin"], title="Daily Price Range")
fig.show()
plt.hist(daily_price_data["drops"],bins=10)
plt.show()

print(hist_hourly_df,hist_hourly_df["Count"].sum())
plt.hist(hourly_price_data["drops"],bins=10)
plt.show()
fig = px.scatter(x=hourly_price_data.index, y=hourly_price_data["drops_bin"], title="Hourly Price Range")
fig.show()


In [29]:
# Compute Collateral Mu and Sigma
df = load_eth_price_data()

# Calculate daily returns
df['datetime'] = pd.to_datetime(df['unix_timestamp'], unit='s')
df.set_index('datetime', inplace=True)
df_daily = df[["unix_timestamp","price_actual"]].resample('D').mean()

# Assuming df_daily is your DataFrame with daily prices in a column named 'value'
# Calculate daily returns
df_daily['daily_return'] = np.log(df_daily['price_actual'] / df_daily['price_actual'].shift(1))

# Drop NaN values (the first entry will be NaN due to the shift)
df_daily = df_daily.dropna()

# Calculate mean (mu) and standard deviation (sigma) of daily returns
mu = df_daily['daily_return'].mean()
sigma = df_daily['daily_return'].std()

print("Mean (mu) of daily returns:", mu)
print("Standard deviation (sigma) of daily returns:", sigma)

fig = px.line(x=df_daily.index, y=df_daily["price_actual"], title="Daily Price")
fig.show()

Mean (mu) of daily returns: 8.569081760129549e-05
Standard deviation (sigma) of daily returns: 0.022516670770215422


/opt/homebrew/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [52]:
# Compute Stable Mu and Sigma
df = load_stable_data()

# Calculate daily returns
df['datetime'] = pd.to_datetime(df['unix_timestamp'], unit='s')
df.set_index('datetime', inplace=True)
df.sort_index(inplace=True)
df_daily = df[["unix_timestamp","price_actual"]].resample('D').mean()

# Assuming df_daily is your DataFrame with daily prices in a column named 'value'
# Calculate daily returns
df_daily['daily_return'] = np.log(df_daily['price_actual'] / df_daily['price_actual'].shift(1))

# Drop NaN values (the first entry will be NaN due to the shift)
df_daily = df_daily.dropna()

# Calculate mean (mu) and standard deviation (sigma) of daily returns
mu = df_daily['daily_return'].mean()
sigma = df_daily['daily_return'].std()

print("Mean (mu) of daily returns:", mu)
print("Standard deviation (sigma) of daily returns:", sigma)

fig = px.line(x=df_daily.index, y=df_daily["price_actual"], title="Daily Price")
fig.show()

print(max(abs(df_daily["daily_return"])))

recovery_times = []

jump_threshold = .04
jump_state = False
jump_timer=0
recovery_mean = 1
recovery_threshold = 0.005

for index, row in df_daily.iterrows():
    # if abs(row["price_actual"]-recovery_mean)>jump_threshold or jump_state==True:
    #     print(index,row["price_actual"]-recovery_mean,jump_state,jump_timer)

    if abs(row["price_actual"]-recovery_mean)>jump_threshold and jump_state==False:
        jump_state=True
        jump_timer=0

    if jump_state==True:
        if abs(recovery_mean-row["price_actual"])<recovery_threshold:
            recovery_times+=[jump_timer]
            jump_timer=0
            jump_state=False
        else:
            jump_timer+=1

print(recovery_times)

    

Mean (mu) of daily returns: -7.048389410022259e-07
Standard deviation (sigma) of daily returns: 0.0030879241521733092


/opt/homebrew/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



0.04599731563062027
2023-03-11 00:00:00 -0.046159756781584016 False 0
2023-03-12 00:00:00 -0.042776040608076826 True 1
2023-03-13 00:00:00 -0.010621570125902258 True 2
2023-03-14 00:00:00 -0.0053682456932243605 True 3
2023-03-15 00:00:00 -0.00444184854501295 True 4
[4]
